The Lagrangian Averaged Vorticity Deviation (LAVD) is computed from the vorticity $ \omega $ as:

\begin{equation}
    \mathrm{LAVD}_{t_0}^{t_N}(\mathbf{x}_0) = \dfrac{1}{t_N-t_0} \int_{t_0}^{t_N}| \omega(\mathbf{x}(t)) - \overline{\omega}(t) |dt \approx 
\dfrac{1}{t_N-t_0} \left [\dfrac{|\omega(\mathbf{x}(t_0)) - \overline{\omega}(t_0) | \Delta t }{2} + \dfrac{|\omega(\mathbf{x}(t_N)) - \overline{\omega}(t_N) | \Delta t }{2} + \sum_{i = 1}^{N-1}| \omega(\mathbf{x}(t_i)) - \overline{\omega}(t_i) | \Delta t \right]
\end{equation}

where $ \omega(\mathbf{x}(t_i)) $ indicates the vorticity at $ \mathbf{x}(t_i) $. $ \overline{\omega}(t) $ is the spatial average of the vorticity at time $ t $.

\begin{equation}
    \mathbf{\omega}(\mathbf{x},t) = \nabla \times \mathbf{v}(\mathbf{x},t) \tag{1}
\end{equation}

| Name | Type (Shape) | Description |
| --- | --- | --- |
| omega | array (Ny, Nx, Nt) | vorticity $ \omega $ along trajectories|
| times | array (Nt, )| time array, equispaced array is assumed|
| LAVD | array (Ny, Nx) | $ \mathrm{LAVD}_{t_0}^{t_N}(\mathbf{x}_0) $|

In [1]:
import sys, os

# get current directory
path = os.getcwd()

# get parent directory
parent_directory = os.path.sep.join(path.split(os.path.sep)[:-3])

# add utils folder to current working path
sys.path.append(parent_directory+"/subfunctions/utils")

In [ ]:
# Import numpy
import numpy as np

# Import function which calculates gradient of velocity
from ipynb.fs.defs.gradient_velocity import gradient_velocity

# Import fast scientific computing library
from numba import njit, prange

In [2]:
@njit(parallel=True)
def _LAVD(omega, times):
    
    ''' 
    The Lagrangian Averaged Vorticity Deviation (LAVD) is computed from the vorticity with the trapezoid rule.
    Integrate the absolute deviation of the vorticity from its spatial mean along trajectories
    and divide by the length of the time interval.
    
    Parameters:
        omega: array(Nt, Ny, Nx), the vorticity vector computed along trajectories
        times: array(Nt, ), time array. Uniform spacing is assumed
        
    Returns:
        LAVD: array(Ny,Nx), integrated |\omega - average(\omega)| / t_N - t_0, the LAVD field
    '''
    
    # Initiliaze arrays and define some variables
    LAVD = np.zeros((omega.shape[1], omega.shape[2])) # array (Ny, Nx)
    lenT = times[-1] - times[0] # calculate length of time interval
    dt = times[1] - times[0] # assume uniform dt
    
    # Compute spatial average of the vorticity
    omega_avg = [] # list (Nt,)
    for t in range(omega.shape[0]):
        omega_avg.append(np.nanmean(omega[t,:,:].ravel())) 
    
    # Compute LAVD through the above integration formula
    for i in prange(omega.shape[1]):
        for j in prange(omega.shape[2]):
            for t in prange(omega.shape[0]):
                
                if t == 0 or t == omega.shape[0]-1:
                    
                    scaling = 1/2
                    
                else: 
                    
                    scaling = 1
                    
                LAVD[i,j] += scaling*abs(omega[t,i,j]-omega_avg[t]) * dt
    
    return LAVD/lenT